In [6]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import yaml
import random
from pathlib import Path
import onnxruntime as ort
import time


In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
import random
import shutil
import glob
from pathlib import Path
import yaml

In [8]:
from ultralytics import YOLO
import onnxruntime as ort
from openvino import Core
import torch

In [9]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import yaml
import random
from pathlib import Path
import ncnn
import time

# Eval

In [10]:
# Hàm benchmark PyTorch
def evaluate_model(model_path, data_yaml, device="cpu", eval_type="Self", imgsz=640, batch=4, num_threads=8, model_name=None):
    print(f"\n📈 [{eval_type}] Evaluating (PyTorch): {os.path.basename(model_path)} on {data_yaml}")

    # --- Cố định threading ---
    os.environ["OMP_NUM_THREADS"] = str(num_threads)
    os.environ["MKL_NUM_THREADS"] = str(num_threads)
    torch.set_num_threads(num_threads)

    # --- Load model YOLO ---
    model = YOLO(model_path)
    model.to(device)

    # --- Validation phase ---
    t0 = time.time()
    metrics = model.val(data=data_yaml, imgsz=imgsz, batch=batch, device=device, verbose=False)
    eval_time = time.time() - t0

    # --- Extract YOLO metrics ---
    map50 = round(metrics.box.map50, 4)
    map95 = round(metrics.box.map, 4)
    prec = round(metrics.box.mp, 4)
    recall = round(metrics.box.mr, 4)
    f1 = round(2 * (prec * recall) / (prec + recall + 1e-9), 4)
    acc = round((prec + recall) / 2, 4)

    # --- Benchmark inference ---
    dummy = torch.randn(1, 3, imgsz, imgsz).to(device)
    warmup, runs = 5, 20
    for _ in range(warmup):
        _ = model.model(dummy)
    t_inf = time.time()
    for _ in range(runs):
        _ = model.model(dummy)
    torch.cuda.synchronize() if device == "cuda" else None
    infer_time = (time.time() - t_inf) / runs
    fps = round(1.0 / infer_time, 2)

    return {
        "Model Name": model_name,
        "Backend": "PyTorch",
        "Eval Type": eval_type,
        "Precision": prec,
        "Recall": recall,
        "F1-score": f1,
        "Accuracy": acc,
        "mAP@50": map50,
        "mAP@50-95": map95,
        "Eval Time (s)": round(eval_time, 2),
        "Infer Time (s/img)": round(infer_time, 4),
        "FPS": fps
    }

# Hàm benchmark ONNX
def evaluate_onnx(onnx_path, data_yaml, eval_type="Self", device="cpu", imgsz=640, batch=4, num_threads=8, model_name=None):
    print(f"\n📈 [{eval_type}] Evaluating (ONNX): {os.path.basename(onnx_path)} on {data_yaml}")

    # --- Setup threading & provider ---
    os.environ["OMP_NUM_THREADS"] = str(num_threads)
    os.environ["MKL_NUM_THREADS"] = str(num_threads)
    providers = ["CPUExecutionProvider"] if device == "cpu" else ["CUDAExecutionProvider"]

    # --- Load YOLO ONNX for evaluation ---
    model = YOLO(onnx_path, task="detect")

    # --- Validation phase ---
    t0 = time.time()
    metrics = model.val(data=data_yaml, split="val", imgsz=imgsz, batch=batch, device=device, verbose=False)
    eval_time = time.time() - t0

    # --- Extract metrics ---
    prec = round(metrics.results_dict.get("metrics/precision(B)", 0), 4)
    recall = round(metrics.results_dict.get("metrics/recall(B)", 0), 4)
    map50 = round(metrics.results_dict.get("metrics/mAP50(B)", 0), 4)
    map5095 = round(metrics.results_dict.get("metrics/mAP50-95(B)", 0), 4)
    f1 = round(2 * (prec * recall) / (prec + recall + 1e-9), 4)
    acc = round((prec + recall) / 2, 4)

    # --- Benchmark inference using onnxruntime ---
    sess_options = ort.SessionOptions()
    sess_options.intra_op_num_threads = num_threads
    sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
    session = ort.InferenceSession(onnx_path, sess_options, providers=providers)
    

    input_name = session.get_inputs()[0].name
    dummy = np.random.randn(1, 3, imgsz, imgsz).astype(np.float32)

    # Warmup + benchmark
    warmup, runs = 5, 20
    for _ in range(warmup):
        _ = session.run(None, {input_name: dummy})
    t_inf = time.time()
    for _ in range(runs):
        _ = session.run(None, {input_name: dummy})
    infer_time = (time.time() - t_inf) / runs
    fps = round(1.0 / infer_time, 2)

    return {
        "Model Name": model_name,
        "Backend": "ONNX",
        "Eval Type": eval_type,
        "Precision": prec,
        "Recall": recall,
        "F1-score": f1,
        "Accuracy": acc,
        "mAP@50": map50,
        "mAP@50-95": map5095,
        "Eval Time (s)": round(eval_time, 2),
        "Infer Time (s/img)": round(infer_time, 4),
        "FPS": fps

    }

# Hàm benchmark OpenVINO
def evaluate_openvino(openvino_path, data_yaml, eval_type="Self", device="cpu", imgsz=640, batch=1, num_threads=4, model_name=None):
    """
    Benchmark OpenVINO model (optimized for Raspberry Pi 5 ARM64)
    
    Args:
        openvino_path: Path to folder containing .xml, .bin files
        data_yaml: Path to dataset YAML
        eval_type: "Self" or "Cross"
        device: "cpu" only (OpenVINO on Pi doesn't support GPU)
        imgsz: Input image size
        batch: Batch size (recommend 1 for Pi)
        num_threads: Number of CPU threads (4 for Pi 5)
    """
    print(f"\n📈 [{eval_type}] Evaluating (OpenVINO): {os.path.basename(openvino_path)} on {data_yaml}")
    
    try:
        from openvino.runtime import Core
    except ImportError:
        print("❌ OpenVINO not installed! Install: pip install openvino")
        return None

    # --- Setup threading ---
    os.environ["OMP_NUM_THREADS"] = str(num_threads)
    os.environ["MKL_NUM_THREADS"] = str(num_threads)
    
    # --- Find .xml file in folder ---
    openvino_folder = Path(openvino_path)
    xml_files = list(openvino_folder.glob("*.xml"))
    
    if not xml_files:
        print(f"❌ No .xml file found in {openvino_path}")
        return None
    
    model_xml = str(xml_files[0])
    print(f"📦 Found model: {model_xml}")
    
    # --- Load YOLO OpenVINO for evaluation ---
    try:
        model = YOLO(openvino_path, task="detect")
    except Exception as e:
        print(f"⚠️ YOLO direct load failed: {e}")
        print("📝 Will use OpenVINO Core directly for inference benchmark")
        model = None
    
    # --- Validation phase (if YOLO wrapper works) ---
    eval_time = 0
    prec = recall = map50 = map5095 = f1 = acc = 0
    
    if model is not None:
        try:
            t0 = time.time()
            metrics = model.val(
                data=data_yaml, 
                split="val", 
                imgsz=imgsz, 
                batch=batch, 
                device=device, 
                verbose=False
            )
            eval_time = time.time() - t0
            
            # --- Extract metrics ---
            prec = round(metrics.results_dict.get("metrics/precision(B)", 0), 4)
            recall = round(metrics.results_dict.get("metrics/recall(B)", 0), 4)
            map50 = round(metrics.results_dict.get("metrics/mAP50(B)", 0), 4)
            map5095 = round(metrics.results_dict.get("metrics/mAP50-95(B)", 0), 4)
            f1 = round(2 * (prec * recall) / (prec + recall + 1e-9), 4)
            acc = round((prec + recall) / 2, 4)
            print(f"✅ Validation completed: mAP@50={map50}")
        except Exception as e:
            print(f"⚠️ Validation failed: {e}")
            print("📝 Skipping validation, will only benchmark inference speed")
    
    # --- Benchmark inference using OpenVINO Core ---
    print("🔧 Loading OpenVINO model for inference benchmark...")
    ie = Core()
    
    # ARM optimization for Pi 5
    config = {
        "PERFORMANCE_HINT": "LATENCY",  # Optimize for low latency
        "NUM_STREAMS": "1",
        "INFERENCE_NUM_THREADS": str(num_threads),
        "ENABLE_CPU_PINNING": "YES",
        "INFERENCE_PRECISION_HINT": "f32"  # Pi 5 doesn't have good fp16 support
    }
    
    compiled_model = ie.compile_model(model=model_xml, device_name="CPU", config=config)
    infer_request = compiled_model.create_infer_request()
    
    # Get input tensor info
    input_layer = compiled_model.input(0)
    input_shape = input_layer.shape
    print(f"📐 Model input shape: {input_shape}")
    
    # Prepare dummy input
    dummy = np.random.randn(1, 3, imgsz, imgsz).astype(np.float32)
    
    # Warmup
    warmup, runs = 5, 20
    print(f"🔥 Warming up {warmup} iterations...")
    for _ in range(warmup):
        infer_request.infer({0: dummy})
    
    # Benchmark
    print(f"⏱️ Benchmarking {runs} iterations...")
    t_inf = time.time()
    for _ in range(runs):
        infer_request.infer({0: dummy})
    infer_time = (time.time() - t_inf) / runs
    fps = round(1.0 / infer_time, 2)
    print(f"✅ Inference time: {infer_time*1000:.2f} ms/image")
    
    return {
        "Model Name": model_name,
        "Backend": "OpenVINO",
        "Eval Type": eval_type,
        "Precision": prec,
        "Recall": recall,
        "F1-score": f1,
        "Accuracy": acc,
        "mAP@50": map50,
        "mAP@50-95": map5095,
        "Eval Time (s)": round(eval_time, 2),
        "Infer Time (s/img)": round(infer_time, 4),
        "FPS": fps
    }

def evaluate_ncnn(ncnn_folder, data_yaml, eval_type="Self", imgsz=640, num_threads=4, model_name=None):
    """
    Benchmark NCNN model (inference bằng runtime thuần NCNN, val dùng YOLO wrapper)
    """
    print(f"\n📈 [{eval_type}] Evaluating (NCNN): {os.path.basename(ncnn_folder)} on {data_yaml}")

    # --- Load model YOLO (wrapper) để tính mAP ---
    try:
        model = YOLO(ncnn_folder, task="detect")
        t0 = time.time()
        metrics = model.val(data=data_yaml, imgsz=imgsz, split="val", batch=1, device="cpu", verbose=False)
        eval_time = time.time() - t0

        prec = round(metrics.results_dict.get("metrics/precision(B)", 0), 4)
        recall = round(metrics.results_dict.get("metrics/recall(B)", 0), 4)
        map50 = round(metrics.results_dict.get("metrics/mAP50(B)", 0), 4)
        map5095 = round(metrics.results_dict.get("metrics/mAP50-95(B)", 0), 4)
        f1 = round(2 * (prec * recall) / (prec + recall + 1e-9), 4)
        acc = round((prec + recall) / 2, 4)
        print(f"✅ Validation done: mAP@50={map50}")
    except Exception as e:
        print(f"⚠️ Validation failed with YOLO wrapper: {e}")
        prec = recall = map50 = map5095 = f1 = acc = 0
        eval_time = 0

    # --- Benchmark inference bằng NCNN thuần ---
    print("🔧 Benchmarking raw NCNN inference...")
    param_path = Path(ncnn_folder) / "model.ncnn.param"
    bin_path = Path(ncnn_folder) / "model.ncnn.bin"
    if not param_path.exists() or not bin_path.exists():
        print(f"❌ Missing NCNN files in {ncnn_folder}")
        return None

    net = ncnn.Net()
    net.opt.use_vulkan_compute = False
    net.opt.num_threads = num_threads
    net.load_param(str(param_path))
    net.load_model(str(bin_path))

    # --- Dummy input ---
    dummy = np.random.randn(1, 3, imgsz, imgsz).astype(np.float32)

    # --- Warmup ---
    warmup, runs = 5, 20
    for _ in range(warmup):
        ex = net.create_extractor()
        ex.input("in0", ncnn.Mat(dummy))
        _, _ = ex.extract("out0")  

    # --- Benchmark ---
    t_inf = time.time()
    for _ in range(runs):
        ex = net.create_extractor()
        ex.input("in0", ncnn.Mat(dummy))
        _, _ = ex.extract("out0")
    infer_time = (time.time() - t_inf) / runs
    fps = round(1.0 / infer_time, 2)

    print(f"✅ NCNN Inference time: {infer_time*1000:.2f} ms/image")

    return {
        "Model Name": model_name,
        "Backend": "NCNN",
        "Eval Type": eval_type,
        "Precision": prec,
        "Recall": recall,
        "F1-score": f1,
        "Accuracy": acc,
        "mAP@50": map50,
        "mAP@50-95": map5095,
        "Eval Time (s)": round(eval_time, 2),
        "Infer Time (s/img)": round(infer_time, 4),
        "FPS": fps
    }


In [ ]:
# --- Đường dẫn dataset duy nhất ---
dataset1 = "/home/pi5/TrafficSign/Dataset/Detect/data.yaml"

# --- Định nghĩa model paths ---
models_pt = {
    "YOLOv5n": "/home/pi5/TrafficSign/WeightDetection/yolo5.pt",
    "YOLOv8n": "/home/pi5/TrafficSign/WeightDetection/yolo8.pt",
    "YOLO11n": "/home/pi5/TrafficSign/WeightDetection/yolo11.pt",
}

models_onnx = {
    "YOLOv5n": "/home/pi5/TrafficSign/convert/model/yolov5/yolov5.onnx",
    "YOLOv8n": "/home/pi5/TrafficSign/convert/model/yolov8/yolov8.onnx",
    "YOLO11n": "/home/pi5/TrafficSign/convert/model/yolov11/yolov11.onnx",
}

models_openvino = {
    "YOLOv5n": "/home/pi5/TrafficSign/convert/model/yolov5/yolo5_openvino_model",
    "YOLOv8n": "/home/pi5/TrafficSign/convert/model/yolov8/yolo8_openvino_model",
    "YOLO11n": "/home/pi5/TrafficSign/convert/model/yolov11/yolo11_openvino_model",
}

models_ncnn = {
    "YOLOv5n": "/home/pi5/TrafficSign/convert/model/yolov5/yolo5_ncnn_model",
    "YOLOv8n": "/home/pi5/TrafficSign/convert/model/yolov8/yolo8_ncnn_model",
    "YOLO11n": "/home/pi5/TrafficSign/convert/model/yolov11/yolo11_ncnn_model",
}

results = []

# --- Benchmark PyTorch ---
for model_name, model_path in models_pt.items():
    print(f"\n🚀 Running PyTorch Benchmark for {model_name}")
    results.append(
        evaluate_model(model_path, dataset1, device="cpu", eval_type=f"{model_name}", model_name=model_name)
    )

# --- Benchmark ONNX ---
for model_name, model_path in models_onnx.items():
    print(f"\n🚀 Running ONNX Benchmark for {model_name}")
    results.append(
        evaluate_onnx(model_path, dataset1, eval_type=f"{model_name}", device="cpu", model_name=model_name)
    )

# --- Benchmark OpenVINO ---
for model_name, model_path in models_openvino.items():
    print(f"\n🚀 Running OpenVINO Benchmark for {model_name}")
    result = evaluate_openvino(model_path, dataset1, eval_type=f"{model_name}", device="cpu", batch=1, num_threads=4, model_name=model_name)
    if result:
        results.append(result)

# --- Benchmark NCNN ---
for model_name, model_path in models_ncnn.items():
    print(f"\n🚀 Running NCNN Benchmark for {model_name}")
    result = evaluate_ncnn(model_path, dataset1, eval_type=f"{model_name}", imgsz=640, num_threads=4, model_name=model_name)
    if result:
        results.append(result)

# --- Lưu kết quả ---
df = pd.DataFrame(results)



🚀 Running PyTorch Benchmark for YOLOv5n

📈 [YOLOv5n] Evaluating (PyTorch): yolo5.pt on /home/pi5/TrafficSign/Dataset/Detect/data.yaml
Ultralytics 8.3.227 🚀 Python-3.13.5 torch-2.9.0+cpu CPU (aarch64)
YOLOv5n summary (fused): 84 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 75.6±6.3 MB/s, size: 1003.8 KB)
val: Scanning /home/pi5/TrafficSign/Dataset/Detect/labels/val.cache... 1082 images, 31 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1082/1082 1.7Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 271/271 0.5it/s 9:10<2.0s
                   all       1082       2783      0.932      0.827      0.909       0.66
Speed: 2.8ms preprocess, 447.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /home/pi5/TrafficSign/runs/detect/val23

🚀 Running PyTorch Benchmark for YOLOv8n

📈 [YOLOv8n] Evaluating (PyTorch): yolo8.pt on /home/pi5/TrafficSign/Datase

OSError: Cannot save file into a non-existent directory: '/Results'

In [13]:
save_csv = "/home/pi5/TrafficSign/Results/fair_benchmark_results_single.csv"
df.to_csv(save_csv, index=False)
print(f"\n✅ Saved benchmark results to: {save_csv}")


✅ Saved benchmark results to: /home/pi5/TrafficSign/Results/fair_benchmark_results_single.csv


In [14]:
display(df)

,Model Name,Backend,Eval Type,Precision,Recall,F1-score,Accuracy,mAP@50,mAP@50-95,Eval Time (s),Infer Time (s/img),FPS
0,YOLOv5n,PyTorch,YOLOv5n,0.9322,0.8268,0.8763,0.8795,0.9090,0.6600,552.35,0.3965,2.52
1,YOLOv8n,PyTorch,YOLOv8n,0.9259,0.8300,0.8753,0.8780,0.9127,0.6652,587.73,0.4177,2.39
2,YOLO11n,PyTorch,YOLO11n,0.9325,0.8345,0.8808,0.8835,0.9134,0.6680,571.83,0.4034,2.48
3,YOLOv5n,ONNX,YOLOv5n,0.9322,0.8268,0.8763,0.8795,0.9090,0.6600,309.51,0.3744,2.67
4,YOLOv8n,ONNX,YOLOv8n,0.9139,0.8394,0.8751,0.8767,0.9109,0.6636,260.83,0.4157,2.41
5,YOLO11n,ONNX,YOLO11n,0.9325,0.8345,0.8808,0.8835,0.9134,0.6680,343.78,0.4665,2.14
6,YOLOv5n,OpenVINO,YOLOv5n,0.9258,0.8246,0.8723,0.8752,0.9053,0.6555,158.73,0.1662,6.02
7,YOLOv8n,OpenVINO,YOLOv8n,0.9147,0.8412,0.8764,0.8780,0.9108,0.6575,139.67,0.1530,6.54
8,YOLO11n,OpenVINO,YOLO11n,0.9289,0.8355,0.8797,0.8822,0.9109,0.6649,143.87,0.1797,5.57
9,YOLOv5n,NCNN,YOLOv5n,0.9230,0.8268,0.8723,0.8749,0.9063,0.6550,159.67,0.0774,12.93
